In [2]:
## Data Ingestion

In [3]:
import os

In [4]:
%pwd

'/Users/vaidehipawar/Desktop/mlops/TextSummarizer/research'

In [5]:
os.chdir("../")
%pwd

'/Users/vaidehipawar/Desktop/mlops/TextSummarizer'

## Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path 
    source_URL: Path 
    local_data_file: Path
    unzip_dir: Path 

In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories 



##  Config 

In [15]:
class ConfigurationManager: 
    def __init__(self, config_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([self.config.data_ingestion.root_dir])


        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir, 
            source_URL=config.source_URL, 
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


In [16]:
import os
import urllib.request as request 
import zipfile 
from src.textSummarizer.logging import logger


## Components

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig): 
        self.config = config 
    
    ## downloading the zip file
    def download_file(self): 
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"File is downloaded")

        else: 
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """ 
        zip_file_path: str,
        Extract the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref: 
            zip_ref.extractall(unzip_path)

## Pipeline

In [18]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config = data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-06-25 13:41:38,006: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-25 13:41:38,007: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 13:41:38,009: INFO: common: created directory at: artifacts]
[2025-06-25 13:41:38,009: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-25 13:41:39,704: INFO: 619097867: File is downloaded]
